In [1]:
import os
import sys
import itertools
import random
import glob
import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader 
import torch.nn as nn
import torchvision
from torchvision import transforms
import torchvision.models as models
from icecream import ic

from PIL import Image
from classes.NIH_dataset_class import NIH_dataset
from classes.Metrics_class import Metrics



# Load data

In [2]:
## Hyperparameters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 512

# Select images paths
DATA_DIR = os.path.join(os.getcwd(), 'dataset')
image_paths = glob.glob(os.path.join(DATA_DIR, 'images_*', 'images', '*.png'))

# Load the data frame to run inference
test_df = pd.read_pickle("test_df.pkl")

# Load the model
model = torch.load('ResNet_50 epochs_0.0001 lr_512 batch_size.pth')
model.to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Transforms
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485], [0.229])
])

# Mapping from disease to number
# disease_to_number = {'Cardiomegaly': 0,
#                     'Emphysema': 1,
#                     'Effusion': 2,
#                     'Hernia': 3,
#                     'Infiltration': 4,
#                     'Mass': 5,
#                     'Nodule': 6,
#                     'Atelectasis': 7,
#                     'Pneumothorax': 8,
#                     'Pleural_Thickening': 9,
#                     'Pneumonia': 10,
#                     'Fibrosis': 11,
#                     'Edema': 12,
#                     'Consolidation': 13}

disease_to_number = {'Cardiomegaly': 0,
                    'Emphysema': 1,
                    'Effusion': 2,
                    'No Finding': 3,
                    'Hernia': 4,
                    'Infiltration': 5,
                    'Mass': 6,
                    'Nodule': 7,
                    'Atelectasis': 8,
                    'Pneumothorax': 9,
                    'Pleural_Thickening': 10,
                    'Pneumonia': 11,
                    'Fibrosis': 12,
                    'Edema': 13,
                    'Consolidation': 14}

C:\Users\suvan\AppData\Local\Temp\ipykernel_48804\1808444168.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('ResNet_50 epochs_0.0001 lr_512 batch_si

In [3]:
test_dataset = NIH_dataset(test_df, image_paths, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
def test(test_loader: DataLoader, model: nn.Module, criterion: nn.Module) -> tuple[nn.Module, Metrics]:
    metrics = Metrics(test_mode=True)
    # Testing loop
    with torch.no_grad():
        for inputs, labels in tqdm.tqdm(test_loader, desc="Testing Batches"):

            inputs = inputs.to(device)
            labels = torch.tensor([disease_to_number[label] for label in labels]).to(device)
            
            outputs = model(inputs)
            max_logits, out_lab = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            metrics.compute(labels.cpu(), out_lab.cpu(), loss=loss)
        return model, metrics

In [5]:
model, metrics = test(test_loader=test_loader, model=model, criterion=criterion)

print(metrics)
# metrics.plot(show=True)

Testing Batches: 100%|██████████| 56/56 [05:50<00:00,  6.26s/it]

            Avg loss: 3.2106
                    Avg accuracy: 0.3725
                    Avg precision: 0.1231
                    Avg recall: 0.1218
                    Avg f1: 0.1201
